<a href="https://colab.research.google.com/github/ro728/AIML-Pupil-Detection/blob/master/Video2X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Video2X](https://github.com/k4yt3x/video2x) Colab Notebook

If you encounter any issues or have any questions or suggestions, post an issue [here](https://github.com/k4yt3x/video2x/issues/new).

Licensed under the ISC License: https://www.isc.org/licenses/. \
Copyright 2022-2023 K4YT3X and contributors.

Last updated on September 28, 2023 for version 5.0.0-beta7, commit `b382f39`.

In [ ]:
#@title ## Step 0: Runtime Selection

#@markdown **You'll need to select a GPU runtime for Video2X to work.** On the menu bar at the top, select:

#@markdown > Runtime > Change runtime type > Hardware accelerator

#@markdown Change "Hardware accelerator" to GPU and click "Save." After you're done, execute this cell to see the detailed information about the GPU allocated to you (optional).

!nvidia-smi

In [ ]:
#@title ## Step 1: Select Algorithms Needed

#@markdown **Run this block and select which algorithms you would like to use.**

#@markdown - Select only what you need. Selecting more algorithms significantly increases installation time.
#@markdown - Selecting `all` installs all algorithms, but it will take a long time (~30 minutes).
#@markdown - You can always rerun steps 1-3 later to add support for more algorithms.

from enum import Enum

import ipywidgets as widgets
from IPython.display import display


class Algorithms(Enum):
    ALL = "all"
    WAIFU2X = "waifu2x"
    SRMD = "srmd"
    REALSR = "realsr"
    RIFE = "rife"
    REALCUGAN = "realcugan"
    ANIME4K = "anime4k"


selected_algorithms = []


def confirm_choice(b):
    global selected_algorithms
    selected_algorithms = []

    if checkboxes[0].value:
        selected_algorithms.append(Algorithms.ALL)
        if any(
            [box.value for box in checkboxes[1:]]
        ):
            print("Warning: 'all' selected, overriding other selections.")
    else:
        for box, algorithm in zip(checkboxes[1:], list(Algorithms)[1:]):
            if box.value:
                selected_algorithms.append(algorithm)

    print(
        f"Selection confirmed: {', '.join([algo.value for algo in selected_algorithms])}"
    )


checkboxes = [
    widgets.Checkbox(value=False, description=algorithm.value)
    for algorithm in Algorithms
]
confirm_button = widgets.Button(description="Confirm")
confirm_button.on_click(confirm_choice)
ui = widgets.VBox(checkboxes + [confirm_button])
display(ui)

In [ ]:
#@title ## Step 2: Install Video2X and Dependencies

#@markdown **This step installs Video2X and its dependencies.**

#@markdown Since some dependencies must be compiled from source code, it may take a while to complete.

# get the optional dependencies to install
if 'selected_algorithms' not in globals() or selected_algorithms is None:
    raise NameError("No algorithms selected. Did you forget to run the previous step?")
algorithms = ','.join([a.value for a in selected_algorithms])

!apt-get update
!apt-get install -y --no-install-recommends \
    python3-pip libvulkan-dev glslang-dev glslang-tools \
    build-essential swig ninja-build ffmpeg nvidia-driver-535 \
    mpv xserver-xorg-video-dummy xvfb

# temporary: install video2x directly from GitHub
#   since version 5.0.0 doesn't have a release on PyPI yet
!pip install -U pip wheel pdm-pep517 setuptools setuptools-scm
!pip install -Uv 'video2x[{algorithms}]@git+https://github.com/k4yt3x/video2x.git@b382f39'

In [ ]:
#@title ## Step 3: Restart Runtime

#@markdown **The runtime must be restarted after installing Video2X** to reload the kernel drivers.

#@markdown Execute this cell to restart the runtime. It's normal that you get an error saying the session crashed.

exit()

In [ ]:
#@title ## Step 4: Select/Upload Input File

#@markdown **Select or upload the file to process.**

#@markdown Google Drive will be used for storage so it will be mounted in this step.

import pathlib

import ipywidgets as widgets
from google.colab import drive, files
from IPython.display import display

drive.mount("/content/drive")
gdrive_path = pathlib.Path("/content/drive/MyDrive")

input_file = None

def on_dropdown_change(change):
    global input_file
    if change["name"] == "value":
        input_file = pathlib.Path(change["new"])
        print(f"Selected input file: {input_file}")

def upload_new_file(button):
    global input_file
    uploaded = files.upload()

    # Check if the user canceled the upload
    if not uploaded:
        print("Upload canceled.")
        return

    for file_name in uploaded.keys():
        input_file = pathlib.Path("/content") / file_name
        print(f"Uploaded and set input file: {input_file}")

file_dropdown = widgets.Dropdown(
    options=list(gdrive_path.iterdir()), description="Select file:"
)
file_dropdown.observe(on_dropdown_change)
display(file_dropdown)

upload_button = widgets.Button(description="Upload New File")
upload_button.on_click(upload_new_file)
display(upload_button)

In [ ]:
#@title ## Step 5.1: Upscale

#@markdown **Fill in the form and run this cell to upscale a video.**

#@markdown - ⚠️ Remember free sessions can run for [at most 12 hours](https://research.google.com/colaboratory/faq.html#idle-timeouts). If you exceed this limit your progress could be lost. You can get [Colab Pro/Pro+](https://colab.research.google.com/signup/pricing) for longer runtimes.
#@markdown - ⚠️ There is a slight chance that the processing gets stuck at `Writing video trailer` for a long time after the processing has completed for an unknown reason. If this happens, check to make sure CPU and GPU usage are at 0%, then hit the top button on the block to stop processing. The output video should be intact.
#@markdown - You can fill in only width or height and the other one will be calculated automatically if it is 0.
#@markdown - Use Anime4K for cartoon-looking videos, and RealSR for real-life footage. RealSR is REALLY slow compared to Anime4K and waifu2x.
#@markdown - It may be a good idea to upscale a small sample of your video to see how well you like the results before processing the whole video.

import contextlib
import os
import pathlib
import subprocess
import sys
import time

import ipywidgets as widgets
import psutil
from google.colab import drive, files
from IPython.display import clear_output, display
from video2x import Video2X

# launch dummy Xorg server for MPV GPU
# write Xorg config
dummy_xorg_conf = """
Section "Device"
    Identifier  "Configured Video Device"
    Driver      "dummy"
EndSection

Section "Monitor"
    Identifier  "Configured Monitor"
EndSection

Section "Screen"
    Identifier  "Default Screen"
    Monitor     "Configured Monitor"
    Device      "Configured Video Device"
EndSection
"""
with open('/etc/X11/xorg.conf', 'w') as f:
    f.write(dummy_xorg_conf)

# start Xorg
for proc in psutil.process_iter():
    with contextlib.suppress(psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
        if proc.name() == 'Xorg':
            print("Killing existing Xorg process")
            proc.kill()
            time.sleep(0.5)
print("Starting dummy Xorg")
subprocess.Popen(["Xorg", ":0"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# export environment variable for subprocesses
%env DISPLAY=:0

if 'input_file' not in globals() or input_file is None:
    raise NameError("Input file not specified. Did you forget to run the previous step?")

if 'gdrive_path' not in globals():
    raise NameError("Google Drive path not specified. Did you forget to run the previous step?")

#@markdown ### Basic Options

# The algorithm to use. Refer to the [Wiki page](https://github.com/k4yt3x/video2x/wiki/Algorithms) for more information.
algorithm = "anime4k" #@param ["anime4k", "waifu2x", "srmd", "realsr", "realcugan"]

# Width and height of the output video. You can fill in one of them and the other one will be calculated automatically if it is left as `0`.
width = 0 #@param {type:"number"}
height = 2160 #@param {type:"number"}

#@markdown ### Advanced Options

#@markdown You typically don't need to touch these if you don't know what they do.

# The de-noise level. Different models support different values. If setting the value result in an error, try lowering the value.
noise = 3 #@param {type:"slider", min:0, max:3, step:1}

# Number of parallel processes to launch. There usually isn't a point setting this higher than 3 as the performance wouldn't improve.
processes = 1 #@param {type:"slider", min:1, max:10, step:1}

# Adjacent frames with < n% diff won't be processed.
threshold = 0 #@param {type:"slider", min:0, max:100, step:1}

if width == 0 and height == 0:
    raise ValueError("You must specify either output width or height")

output_file = gdrive_path / f"{input_file.stem}_{algorithm}_{width}x{height}{input_file.suffix}"
if output_file.exists():
    raise FileExistsError(f"Cannot continue: file {output_file} already exists")
print(f"Output will be written to: {output_file}")

video2x = Video2X()
video2x.upscale(
    input_file,
    output_file,
    width,
    height,
    noise,
    processes,
    threshold,
    algorithm,
)

print("Video processing completed successfully")

In [ ]:
#@title ## Step 5.2: Interpolate (🚧 WIP, NOT WORKING YET 🚧)

#@markdown Fill in the form and run this cell to interpolate a video (currently only 2x is supported). **Remember to use the free resource fairly and not to run a free session for over 12 hours.** You can get [Colab Pro/Pro+](https://colab.research.google.com/signup/pricing) for longer runtimes.

#@markdown - ⚠️ While working fine locally, still doesn't seem to work right in Colab.
#@markdown - Decrease the `threshold` if you notice smearing between scenes.
#@markdown - Increase the `threshold` if you notice frame drops in one scene.
#@markdown - You don't need to touch `processes` unless you know what it does.

import pathlib
from video2x import Video2X

video2x = Video2X()

# The input and output files' file names. Video2X will operate in the root directory of your Google Drive.
input_filename = "input.mp4" #@param {type:"string"}
output_filename = "output.mp4" #@param {type:"string"}

# Number of parallel processes to launch. There usually isn't a point setting this higher than 3 as the performance wouldn't improve.
processes = 1 #@param {type:"slider", min:1, max:10, step:1}

# Adjacent frames with > n% diff won't be processed.
threshold = 5 #@param {type:"slider", min:0, max:100, step:1}

# The algorithm to use. Refer to the [Wiki page](https://github.com/k4yt3x/video2x/wiki/Algorithms) for more information.
algorithm = "rife" #@param ["rife"]

# check if input file exists
gdrive = pathlib.Path("/mnt/gdrive/MyDrive/")
if not (gdrive / input_filename).is_file():
    raise FileNotFoundError(f"file {(gdrive / input_filename)} does not exist")

video2x.interpolate(
    gdrive / input_filename,
    gdrive / output_filename,
    processes,
    threshold,
    algorithm,
)